<a href="https://colab.research.google.com/github/satani99/prosody_clone/blob/main/Toucan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/satani99/IMS-Toucan.git

Cloning into 'IMS-Toucan'...
remote: Enumerating objects: 16681, done.
remote: Counting objects: 100% (5477/5477), done.
remote: Compressing objects: 100% (1210/1210), done.
remote: Total 16681 (delta 4408), reused 4275 (delta 4266), pack-reused 11204
Receiving objects: 100% (16681/16681), 12.27 MiB | 17.26 MiB/s, done.
Resolving deltas: 100% (12603/12603), done.


In [2]:
%cd IMS-Toucan

/content/IMS-Toucan


In [3]:
!pip install alias_free_torch auraloss cvxopt deepfilternet dragonmapper gradio jinja2 librosa matplotlib==3.4.3 numba numpy pedalboard phonemizer praat-parselmouth pyloudnorm pypinyin scikit-learn scipy sounddevice soundfile speechbrain torch torchaudio torch_complex tqdm urllib3 wandb six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.9/37.9 MB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.8/553.8 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.1/544.1 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.5 MB/s eta 0:00:

In [4]:
import os
import numpy as np
import soundfile
import soundfile as sf
import torch
from tqdm import tqdm

In [5]:
!sudo apt-get install espeak-ng
!sudo apt-get install libportaudio2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
The following NEW packages will be installed:
  espeak-ng espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 8 not upgraded.
Need to get 4,525 kB of archives.
After this operation, 11.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpcaudio0 amd64 1.1-6build2 [8,956 B]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsonic0 amd64 0.2.0-11build1 [10.3 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 espeak-ng-data amd64 1.50+dfsg-10 [3,956 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 libespeak-ng1 amd64 1.50+dfsg-10 [207 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 espeak-ng amd64 1.50+dfsg-10 [343 kB]
Fetched 4,525 kB in 0s (10.8 MB/

In [6]:
from run_model_downloader import download_models
download_models()

[..................................................][██................................................][███...............................................][█████.............................................][███████...........................................][█████████.........................................][███████████.......................................][█████████████.....................................][███████████████...................................][███████████

In [7]:
from Preprocessing.TextFrontend import ArticulatoryCombinedTextFrontend
from Preprocessing.AudioPreprocessor import AudioPreprocessor
from TrainingInterfaces.Text_to_Spectrogram.AutoAligner.Aligner import Aligner
from TrainingInterfaces.Text_to_Spectrogram.FastSpeech2.DurationCalculator import DurationCalculator
from InferenceInterfaces.UtteranceCloner import UtteranceCloner
from Preprocessing.articulatory_features import get_feature_to_index_lookup

In [8]:
def float2pcm(sig, dtype="int16"):
  sig = np.asarray(sig)
  if sig.dtype.kind != 'f':
    raise TypeError("'sig' must be a float array")
  dtype = np.dtype(dtype)
  dtype = np.dtype(dtype)
  if dtype.kind not in 'iu':
    raise TypeError("'dtype' must be an integer type")
  i = np.iinfo(dtype)
  abs_max = 2 ** (i.bits - 1)
  offset = i.min + abs_max
  return (sig * abs_max + offset).clip(i.min, i.max).astype(dtype)


In [18]:
class TTS_Interface:

  def __init__(self):
    self.device = "cuda" if torch.cuda.is_available() else "cpu"

    self.utterance_cloner = UtteranceCloner(model_id="Meta", device=self.device)
    self.speaker_path_lookup = {"Voice": "download (1).wav"}
    self.acoustic_model = Aligner()
    self.acoustic_model.load_state_dict(torch.load("Models/Aligner/aligner.pt", map_location='cpu')["asr_model"])
    self.acoustic_model = self.acoustic_model.to(self.device)
    self.dc = DurationCalculator(reduction_factor=1)
    self.tf = ArticulatoryCombinedTextFrontend(language="en")
    example_audio, sr = soundfile.read("9.wav")
    self.ap = AudioPreprocessor(input_sr=sr, output_sr=16000, )

    steps = 10
    tokens = list()
    for vector in self.tf.string_to_tensor(
        "Don't forget what they said yesterday,two policemen is excessive."):
      if vector[get_feature_to_index_lookup()["word-boundary"]] == 0:
        for phone in self.tf.phone_to_vector:
          if vector.numpy().tolist()[13:] == self.tf.phone_to_vector[phone][13:]:
            tokens.append(self.tf.phone_to_id[phone])
            break

    tokens = torch.LongTensor(tokens).squeeze().to(self.device)
    tokens_len = torch.LongTensor([len(tokens)]).to(self.device)
    mel = self.ap.audio_to_mel_spec_tensor(example_audio, normalize=True).transpose(0, 1).unsqueeze(0).to(self.device)
    mel.requires_grad = True
    mel_len = torch.LongTensor([len(mel[0])]).to(self.device)
    optim_asr = torch.optim.SGD(self.acoustic_model.parameters(), lr=0.1)
    self.acoustic_model.train()
    for _ in tqdm(list(range(steps))):
      pred =  self.acoustic_model(mel)
      loss = self.acoustic_model.ctc_loss(pred.transpose(0, 1).log_softmax(2), tokens, mel_len, tokens_len)
      optim_asr.zero_grad()
      loss.backward()
      torch.nn.utils.clip_grad_norm_(self.acoustic_model.parameters(), 1.0)
      optim_asr.step()
    self.acoustic_model.eval()

    reference_audio = "9.wav"
    prompt = "Don't forget what they said yesterday,two policeman is excessive."
    text_list = prompt.replace(".", ".|").replace("?", "?|").replace("!", "!|").split("|")

    self.split_audio(reference_audio, text_list)

    self.utterance_cloner.tts.set_utterance_embedding("download (1).wav")
    self.voice = self.utterance_cloner.clone_utterance(path_to_reference_audio_for_intonation="split_1.wav",
                                                       path_to_reference_audio_for_voice="download (1).wav",
                                                       transcription_of_intonation_reference=text_list[0],
                                                       lang="en",
                                                       filename_of_result="output.wav")

  # def read(self, _, speaker):
  #   reference_audio = "9.wav"

  #   if speaker == "Voice":
  #     part_1 = self.voice

  #   return "alignment.png", \
  #          reference_audio, \
  #          self.speaker_path_lookup["Voice"], \
  #          (24000, float2pcm((torch.tensor(part_1)).numpy()))

  def split_audio(self, path_to_audio, text_list):
    audio, sr = sf.read(path_to_audio)
    ap = AudioPreprocessor(input_sr=sr, output_sr=16000, melspec_buckets=80, hop_length=256, n_fft=1024, cut_silence=False)
    norm_wave = ap.audio_to_wave_tensor(normalize=True, audio=audio)
    melspec = ap.audio_to_mel_spec_tensor(audio=norm_wave, normalize=False, explicit_sampling_rate=16000).transpose(0, 1)

    lines = list()
    self.tf.use_word_boundries = False
    for segment in text_list:
      if segment.strip() != "":
        lines.append(self.tf.string_to_tensor(segment, handle_missing=False).squeeze())

    self.tf.use_word_boundries = True

    processed_lines = list()
    for index, line in enumerate(lines):
      if index == 0:
        processed_lines.append(line[:-1])
      else:
        processed_lines.append(line[1:-1])
    lines = processed_lines
    joined_phonemes = torch.cat(lines, dim=0)

    alignment_paths = list()
    ensemble_of_durations = list()
    for ensemble in range(1):
      alignment_paths.append(self.acoustic_model.inference(mel=melspec.to(self.device),
                                                           tokens=joined_phonemes.to(self.device),
                                                           save_img_for_debug="alignment.png" if ensemble == 0 else None,
                                                           return_ctc=False))

    for alignment_path in alignment_paths:
      ensemble_of_durations.append(self.dc(torch.LongTensor(alignment_path), vis=None).squeeze())
    durations = list()

    for i, _ in enumerate(ensemble_of_durations[0]):
      duration_of_phone = list()
      for ensemble_member in ensemble_of_durations:
        duration_of_phone.append(ensemble_member.squeeze()[i])
      durations.append(sum(duration_of_phone) / len(duration_of_phone))

    line_lens = [len(x) for x in lines]
    index = 0
    segment_durations = list()
    for num_phones in line_lens:
      segment_durations.append(sum(durations[index: index + num_phones]))
      index += num_phones
    spec_to_wave_factor = len(norm_wave) / sum(segment_durations)
    wave_segment_lens = [int(x * spec_to_wave_factor) for x in segment_durations]
    start_index = 0
    wave_segments = list()
    for index, segment_len in enumerate(wave_segment_lens):
      if index == len(wave_segment_lens) - 1:
        wave_segments.append(norm_wave[start_index:])
      else:
        wave_segments.append(norm_wave[start_index: start_index + segment_len])
        start_index += segment_len

    for index, wave_segment in enumerate(wave_segments):
      sf.write(f"split_{index + 1}.wav", wave_segment, 16000)




In [19]:
meta_model = TTS_Interface()

100%|██████████| 10/10 [00:00<00:00, 28.94it/s]
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1501: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:982.)
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1501: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:982.)
  return forward_call(*args, **kwargs)
100%|██████████| 5/5 [00:00<00:00, 24.99it/s]


##Output Audio

In [20]:
import IPython

IPython.display.Audio("output.wav")

##Target Audio

In [28]:
import IPython

IPython.display.Audio("download (1).wav")

##Reference Audio

In [29]:
import IPython

IPython.display.Audio("9.wav")

In [14]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [15]:
!pip install voicefixer==0.1.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.8/203.8 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 82.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 95.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 20.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 113.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 32.7 MB/s eta 0:00:00
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12067 sha256=8279c23b89649301c6d9c02bddb215cb14577499d43b18e15deacaee62394c99
  Stored in directory: /root/.cache/pip/wheels/cd/17/e5/765d1a3112ff3978f70223502f6047e06c43a24d7c5f8ff95b
  Created wh

In [21]:
!voicefixer --infile output.wav --outfile output_up.wav --mode 0

Initializing VoiceFixer
Start processing the input file output.wav.
Processing output.wav, mode=0
Done


##Output Audio Upscaled

In [22]:
import IPython

IPython.display.Audio("output_up.wav")